![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
# заполняем пропусками
data['Number of Reviews'].fillna(0, inplace=True)
data['Reviews'].fillna('[[][]]')

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

In [ ]:
#ratio restraurant's reviews/average reviews in a city
def create_column_ratio_reviews(df_input):
    df_output = df_input.copy()
    cities_avg_rev = df_output.groupby('City')['Number of Reviews'].mean()
    
    def get_ratio(num_rev, avg_value):
        return num_rev/avg_value
    
    df_output['Ratio_Reviews'] =  df_output.apply(lambda x: get_ratio(x['Number of Reviews'], cities_avg_rev[x['City']]), axis = 1)
    return df_output

In [ ]:
data=create_column_ratio_reviews(data)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
#Price Range update
def update_price_range(df_input):
    df_output = df_input.copy()
    price_range_type = {
        '$': 1,
        '$$ - $$$': 2,
        '$$$$': 3,
    }

    df_output['Price Range']=df_output['Price Range'].replace(to_replace=price_range_type)
    #fill nan value to 0
    df_output['Price Range']=df_output['Price Range'].fillna(0)
    
    return df_output

In [ ]:
data = update_price_range(data)

In [ ]:
# this function helps us prepare Cuisine Style column for future work
def convert_cuisines_styles(df_input):
    df_output = df_input.copy()
    df_output['Cuisine Style']=df_output['Cuisine Style'].fillna('')
    df_output['Cuisine Style']=df_output['Cuisine Style'].apply(lambda x:x.replace('[', ''))
    df_output['Cuisine Style']=df_output['Cuisine Style'].apply(lambda x:x.replace(']', ''))
    return df_output

In [ ]:
data = convert_cuisines_styles(data)

In [ ]:
#convert cuisine_style to columns
def add_cuisine_style_columns(df_input):
    df_output = df_input.copy()
    unique_cus_style = set()
    for item in df_output['Cuisine Style']:
        arr= item.split(',')
        unique_cus_style.update(arr)

    unique_cus_style = [x.replace("'","").strip() for x in unique_cus_style if x!=""]
    
    #create columns in data
    for item in unique_cus_style:
        df_output[item]=0

    def set_cuisine(x, elem):
        arr = x.split(',')
        arr_conv=[item.replace("'","").strip() for item in arr if item!=""]
        if elem in arr_conv:
            return 1
        return 0

    for item in unique_cus_style:
        df_output[item]=df_output['Cuisine Style'].apply(set_cuisine, args=(item,)).astype('uint8')
    
    return df_output

In [ ]:
data=add_cuisine_style_columns(data)

In [ ]:
# check if restauran has top cuisine
import collections
def get_top_cuisine_style(df_input):
    df_output = df_input.copy()
    cus_style_count = collections.Counter()
    for item in df_output['Cuisine Style']:
        arr= item.split(',')
        arr = [x.replace("'","").strip() for x in arr if x!=""]
        for elem in arr:
            cus_style_count[elem] +=1

    return cus_style_count.most_common(4)

def add_column_has_top_cuisine(df_input):
    df_output = df_input.copy()
    top_cuisines = get_top_cuisine_style(df_output)
    top_cuisines = [x[0] for x in top_cuisines]
    
    def has_top_cuisines(x, top_cuisines):
        arr = x.split(',')
        arr_conv=[item.replace("'","").strip() for item in arr if item!=""]
        return int(bool(set(arr_conv) & set(top_cuisines)))
    
    df_output['Is top cuisines'] = df_output['Cuisine Style'].apply(has_top_cuisines, args=(top_cuisines,)).astype('uint8')
    return df_output

In [ ]:
data = add_column_has_top_cuisine(data)

In [ ]:
#number of cuisines in restauran
def create_column_number_of_cuisines(df_input):
    df_output = df_input.copy()
    
    def get_number_of_cuisines(x):
        arr = x.split(',')
        return len(arr)
    df_output['Cuisines_Num']=df_output['Cuisine Style'].apply(get_number_of_cuisines).astype('uint8')
    return df_output
    

In [ ]:
data = create_column_number_of_cuisines(data)

In [ ]:
# get dif between first and last review
from datetime import datetime
import re
def create_column_time_dif(df_input):
    df_output = df_input.copy()
    def get_diff(x):
        
        arr = re.findall( '\d{2}\/\d{2}\/\d{4}', str(x), re.I|re.M)
        arrd=[datetime.strptime(x, '%m/%d/%Y') for x in arr]
        if(len(arrd)>1):
            return (max(arrd)-min(arrd)).days
        return 0
    df_output['Time_Review_Diff']=df_output['Reviews'].apply(get_diff)
    return df_output

In [ ]:
data = create_column_time_dif(data)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### Проверка корреляции - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
Из проверка корреляции были исключены столбцы с городами и видами кухонь, в силу того, что их наличие мешала оценке остальных признаков.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data[[ 'Rating', 'Ranking', 'Number of Reviews', 'Price Range', 'Is top cuisines', 'Cuisines_Num', 'Ratio_Reviews', 'Time_Review_Diff']].corr(),)

In [ ]:
corr = data[[ 'Rating', 'Ranking', 'Number of Reviews', 'Price Range', 'Is top cuisines', 'Cuisines_Num', 'Ratio_Reviews', 'Time_Review_Diff']].corr()
corr.style.background_gradient(cmap='coolwarm')

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Reviews'].fillna('[[][]]')   
    df_output['Number of Reviews'].fillna(0, inplace=True)

    # ################### 4. Feature Engineering ####################################################
    # because we use City column to get this parameter, we run it before get_dummies operation
    df_output = create_column_ratio_reviews(df_output)
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # Обработка'Price Range'
    df_output = update_price_range(df_output)
    
    # Prepare Cuisine Style for future work
    df_output = convert_cuisines_styles(df_output)
    # convert cuisine style to colums
    df_output = add_cuisine_style_columns(df_output)
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # has columns top cuisines
    df_output = add_column_has_top_cuisine(df_output)
    # number of cuisines
    df_output = create_column_number_of_cuisines(df_output)
    df_output = create_column_time_dif(df_output)
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
